In [45]:
#dependencies 
import nltk 
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

In [54]:
df=pd.read_csv('D:/Data Sets/fake-news/train.csv') #reading file csv and converting to a dataframe

In [55]:
#first 5 columns
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [56]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [57]:
print(1957*100/df.shape[0])#9 percent can be removed 

9.408653846153847


In [58]:
df=df.dropna()

In [59]:
df.shape

(18285, 5)

In [60]:
#the indices would have disrupted
df.reset_index(inplace=True)

In [ ]:
#label column means the output variable
#0 represents that the news can be relied on(not fake)
#1 represents it cannot be relied on(fake)

In [61]:
#dependent variable(y)
y=df['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
18280    0
18281    0
18282    0
18283    1
18284    1
Name: label, Length: 18285, dtype: int64

In [62]:
#independent variable
x=df.iloc[:,:-1]
x

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [63]:
x['text'][4] # in this we see there are characters like \n which we will be removing, for that we need re(regular expression) 

'Print \nAn Iranian woman has been sentenced to six years in prison after Iran’s Revolutionary Guard searched her home and found a notebook that contained a fictional story she’d written about a woman who was stoned to death, according to the Eurasia Review . \nGolrokh Ebrahimi Iraee, 35, is the wife of political prisoner Arash Sadeghi, 36, who is serving a 19-year prison sentence for being a human rights activist, the publication reported. \n“When the intelligence unit of the Revolutionary Guards came to arrest her husband, they raided their apartment – without a warrant – and found drafts of stories that Ebrahimi Iraee had written,” the article stated. \n“One of the confiscated drafts was a story about stoning women to death for adultery – never published, never presented to anyone,” the article stated. “The narrative followed the story of a protagonist that watched a movie about stoning of women under Islamic law for adultery.'

In [69]:
import re
from nltk.stem import PorterStemmer as ps
from tqdm import tqdm

In [70]:
#here the main text is the key feature because it tells us whether the news can be relied or not,
#alone with headline we cannot determine how reliable the news is
ps=ps()
corpus=[]
l=len(x['text'])
for i in tqdm(range(l)):
    texts=re.sub('[^a-zA-Z]',' ',x['text'][i])
    texts=texts.lower()
    texts=texts.split()
    texts=[ps.stem(text) for text in texts if text not in stopwords.words('english')]
    texts=' '.join(texts)
    corpus.append(texts)
    

100%|██████████████████████████████████████████████████████████████████████████| 18285/18285 [1:29:19<00:00,  3.41it/s]


In [72]:
#it literally took 1:29:19 to pre process the text!!!!
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
tfidf=tfidf(max_features=6000,ngram_range=(1,3))
x=tfidf.fit_transform(corpus).toarray()

In [73]:
x[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [74]:
x.shape

(18285, 6000)

In [75]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [76]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()
classifier.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [77]:
#predicting values
y_pred=classifier.predict(x_test)

In [79]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[2431  120]
 [ 352 1669]]


In [84]:
print(accuracy_score(y_test,y_pred))

0.8967629046369204
